In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Pseudo-code
GOAL: take in RAW, not quality controlled data and clean it up. This does NOT include interpolation of missing data points, but does include:
1. Compare to sensor health data or logs of known failure periods and remove data in those windows
2. Plausible value check. Removes any values that are beyond standard range for each variable
3. Remove noise. Higher-level processing but could follow USGS methods
4. Compare site-best. If multiple sensors exist for a given data variable, compare values and fill gaps 
5. Combine data files. If data is separated into multiple datafiles, resample to get on the same time basis (can resample to hourly here, or the lowest resolution of what is available). Combine files with consistent naming into one dataframe for checks

In [74]:
AWS_fp = '~/research/climate_data/AWS/Raw/'
data_fns = ['SouthGlacier_AWS_HalfHourData.csv','SouthGlacier_AWS_FiveMinData.csv','SouthGlacier_AWS_HealthData.csv']
time_vn = 'TIMESTAMP'

In [95]:
df = pd.read_csv(AWS_fp+data_fns[0])
start = pd.to_datetime(df[time_vn][0])
end = pd.to_datetime(df[time_vn][np.shape(df)[0]-1])
df = df.set_index(pd.to_datetime(df[time_vn]))
df.columns

Index(['TIMESTAMP', 'RECORD', 'DT', 'TCDT', 'BP', 'BPSC', 'Rain_mm_Tot'], dtype='object')

Rename variables to have consistency

*** How to handle when there are multiple columns with the same data? i.e. two SWin terms

In [96]:
names = {'temp':['site_temp_USGS','temperature','Tair_aws','temp','TA_2.0m','T','AirTC'],
            'tp':['Precip_Weighing_Incremental','precipitation','Ptotal_aws','tp','P','Rain_mm_tot'],
            'rh':['RelHum','RH','rh','rH','RH_aws','RH_2.0m'],
            'SWin':['RadiationIn','SWin','SWin_aws','SW_IN','short_dn_Avg'],
            'SWout':['RadiationOut','SWout','SWout_aws','SW_out','SW_OUT','short_up_Avg'],
            'LWin':['LWRadiationIn','LWin','LWin_aws','LW_in','LW_IN','long_dn_corr_Avg'],
            'LWout':['LWRadiationOut','LWout','LWout_aws','LW_OUT','long_up_corr_Avg'],
            'wind':['WindSpeed','wind','Wind','ws_aws','WS','WS_ms_S_WVT'],
            'winddir':['VecAvgWindDir','WindDir','Winddir','winddir','WD','WindDir_D1_WVT'],
            'sp':['barom','sp','press','Press_aws','Barom','BP'],
            'tcc':['cloud_fraction','tcc','CCF','CCF_aws']}
# RENAMING
drop_vars = []
all_vars = ['temp','tp','rh','SWin','SWout','LWin','LWout','wind','winddir','sp','tcc']
for var in df.columns.to_numpy():
    renamed = False
    for var_check in all_vars:
        if var in names[var_check]:
            df = df.rename(columns={var:var_check})
            all_vars.remove(var_check)
            renamed = True
    if not renamed:
        drop_vars.append(var)
if len(drop_vars) > 0:
    print(len(drop_vars),'variables were not renamed, including:')
    print(drop_vars)
else:
    drop_vars = [0]
df = df.drop(columns=drop_vars)
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].astype(float)

6 variables were not renamed, including:
['TIMESTAMP', 'RECORD', 'DT', 'TCDT', 'BPSC', 'Rain_mm_Tot']


Check datatypes to sort out random strings or non-float values

Sensor malfunctions to remove time periods where the sensor is known to be malfunctioning, according to some indicator (panel temperature, voltage, etc.) and healthy limit for said indicator

In [97]:
# Load in health dataset and specify bounds to remove datapoints
health_df = pd.read_csv(AWS_fp+data_fns[2],index_col=time_vn)
indicator = 'Panel_Temp_Max'
healthy_limit = 10
unhealthy_idx = health_df[indicator] > healthy_limit
# print(unhealthy_idx)
print(unhealthy_idx.value_counts())


False    3085
True      251
Name: Panel_Temp_Max, dtype: int64


Plausible values

In [98]:
# Define boundaries for each variable
bounds = {'temp':[-40,30],'precip':[0,60],'wind':[0,75],'winddir':[0,360],
            'sp':[50,110],'SWin':[0,1600],'SWout':[0,1600],'LWin':[-100,400],'LWout':[-100,400],'rh':[0,100],'tcc':[0,100]}
units = {'temp':'C','precip':'mm hr-1','wind':'m s-1','winddir':'deg',
            'sp':'kPa','sw':'W m-2','lw':'W m-2','rh':'%','tcc':'%'}
# df['sp'] = df['sp'] / 10
for var in df.columns:
    over = df[var] > bounds[var][1]
    under = df[var] < bounds[var][0]
    print('over',over.value_counts(),'under',under.value_counts())
    df[var] = df[var].mask(over,bounds[var][1])
    df[var] = df[var].mask(under,bounds[var][0])

over True    160104
Name: sp, dtype: int64 under False    160104
Name: sp, dtype: int64


In [99]:
df_half_hour = df.resample('H').mean()

In [100]:
df_five_min['sp'] = df_half_hour['sp']

In [103]:
df_five_min.to_csv('~/research/climate_data/AWS/Raw/southglacier.csv')

In [94]:
df_five_min = df.resample('H').mean()
print(df_five_min)

                         wind     winddir      temp         rh      SWout  \
TIMESTAMP                                                                   
2006-07-09 15:00:00  2.924545  300.618182  3.542000  76.073636        NaN   
2006-07-09 16:00:00  2.235833  289.091667  3.145000  82.265833        NaN   
2006-07-09 17:00:00  0.574625  184.580000  3.541875  85.627500        NaN   
2006-07-09 18:00:00  0.927150  156.536500  3.652900  88.525000        NaN   
2006-07-09 19:00:00  2.803000  231.638333  3.172000  90.466667        NaN   
...                       ...         ...       ...        ...        ...   
2015-08-27 04:00:00  1.008500   95.405583 -0.140333  86.441667   0.024507   
2015-08-27 05:00:00  1.089500  211.322167 -0.499333  87.383333   0.000000   
2015-08-27 06:00:00  0.936917  152.792167 -0.473917  89.450000   0.586242   
2015-08-27 07:00:00  1.698333  121.810000 -0.029000  87.816667  11.155348   
2015-08-27 08:00:00  1.098667  119.483333  0.306333  84.150000  41.789327   

Multiple data files

In [48]:
# def merge_files(fns):
fns = data_fns[0:2]
df = pd.read_csv(AWS_fp+fns[0],index_col=time_vn)
timestep_original = pd.to_datetime(df.index[1]) - pd.to_datetime(df.index[0])
for fn in fns[1:]:
    df_load = pd.read_csv(AWS_fp+fn,encoding='ISO-8859-1')
    timestep_load = pd.to_datetime(df_load.index[1]) - pd.to_datetime(df_load.index[0])
    if timestep_load < timestep_original:
        print(timestep_load.seconds)

/tmp/ipykernel_14775/361441679.py:6: DtypeWarning: Columns (11,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_load = pd.read_csv(AWS_fp+fn,encoding='ISO-8859-1')


0


In [1]:
def basic_stats(fn,header,droprows):
    df = pd.read_csv(AWS_fp+fn,header=header,encoding = 'ISO-8859-1')
    df = df.drop(droprows,axis=0)
    df = df.set_index(time_vn)
    for column in df.columns:
        print(column,'Nonzero count:',df[column].count(),'          ',df[column].mean())

In [33]:
basic_stats(data2_fn,1,[0,1])

/tmp/ipykernel_2729/916621188.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(AWS_fp+fn,header=header,encoding = 'ISO-8859-1')


RECORD Nonzero count: 956668
WS_ms_S_WVT Nonzero count: 956668
WindDir_D1_WVT Nonzero count: 956668
WindDir_SD1_WVT Nonzero count: 956668
WS_ms_Max Nonzero count: 956668
AirTC Nonzero count: 956668
NR_Wm2_Avg Nonzero count: 956668
CNR_Wm2_Avg Nonzero count: 191532
RH Nonzero count: 956668
SWin_Wm2_Avg Nonzero count: 457951
SWout_Wm2_Avg Nonzero count: 457951
cnr4_T_C_Avg Nonzero count: 307187
short_up_Avg Nonzero count: 307187
short_dn_Avg Nonzero count: 307187
long_up_corr_Avg Nonzero count: 307187
long_dn_corr_Avg Nonzero count: 307187


In [8]:
import pandas as pd
df = pd.read_csv('/home/claire/research/climate_data/AWS/Preprocessed/saintsorlin/saintsorlin_hourly.csv',index_col=0)
print(df.index.to_numpy()[-1])

2021-12-31 23:00:00
